In [1]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt
import gymnasium as gym

#np.set_printoptions(precision=4)
import math
from utils import *

size = 8
nA = 2
seed = get_current_seed()
gamma = .9
mu = np.ones(size) * 1/size
river = River(size, mu, gamma=gamma, small=5, large=1000, seed=seed, render_mode = "human")
tau = 0.5
xi = np.ones(size) * 1/size
tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
#mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)

env = gym.make('FrozenLake-v1', render_mode="human", is_slippery=False)



Current seed for result reproducibility: 52042883736869560754474800480465396903


In [2]:
tmdp = TMDP(env, xi, tau=tau, gamma=gamma, seed=seed)

In [4]:
env.reset()
np_random, seed = seeding.np_random(seed)
np_random.choice(len(mu), p=mu)


AttributeError: accessing private attribute '_get_obs' is prohibited